# A Multi-Agent Workflow

In this lesson, you'll build a data agent that can perform web research, answer questions, and generate charts.

Let's load the environment variables that define the OpenAI API and Tavily API keys.

In [1]:
# Install all required dependencies
%pip install -q python-dotenv
%pip install -q langchain==0.2.0
%pip install -q langchain-openai==0.1.7
%pip install -q langchain-community==0.2.0
%pip install -q tavily-python==0.5.0
%pip install -q langgraph==0.1.0
%pip install -q matplotlib==3.9.2
%pip install -q pandas==2.2.3
%pip install -q seaborn==0.13.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 0.0.8, 0.1.18, 0.2.29, 0.2.30, 0.2.31, 0.3.0, 0.4.4, 0.6.9
ERROR: Could not find a version that satisfies the requirement langgraph==0.1.0 (from versions: 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.57, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.62, 0.0.63, 0.0.64, 0.0.65, 0.0.66, 0.0.67, 0.0.68, 0.0.69, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.1.10, 0.1.11, 0.1.12, 0.1.13, 0.1.14, 0.1.15, 0.1.16, 0.1.17, 0.1.19, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5a0, 0.2.5, 0.2.6, 0.2.7a0, 0.2.7, 0.2.8, 0.2.9, 0.2.10, 0.2.11, 0.2.12, 0.2.13, 0.2.14, 0.2.15, 0.2.16, 0.2.17, 0.2.18, 0.2.19, 0.2.20, 0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os

# Load environment variables
_ = load_dotenv(override=True)

# Optional: Set API keys directly if not using .env file
# os.environ['OPENAI_API_KEY'] = 'your-openai-api-key'
# os.environ['TAVILY_API_KEY'] = 'your-tavily-api-key'

**Note**: These variables are already defined in this environment. If you'd like to run the notebook locally, you can define them in a `.env` file. For an env template, you can check the file `env.template` in this lesson's folder.

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access <code>requirements.txt</code>, <code>env.template</code>, <code>prompts.py</code>, and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>

## 2.1 Initialize the agent's state

State provides the agent with a shared, evolving memory across nodes so that the agents have the context and instructions needed to act coherently and achieve the goal.

In [3]:
from typing import Literal, Optional, List, Dict, Any, Type
from langgraph.graph import MessagesState

# Custom State class with specific keys
class State(MessagesState):
    user_query: Optional[str] # The user's original query
    enabled_agents: Optional[List[str]] # Makes our multi-agent system modular on which agents to include
    plan: Optional[List[Dict[int, Dict[str, Any]]]] # Listing the steps in the plan needed to achieve the goal.
    current_step: int # Marking the current step in the plan.
    agent_query: Optional[str] # Inbox note: `agent_query` tells the next agent exactly what to do at the current step.
    last_reason: Optional[str] # Explains the executor's decision to help maintain continuity and provide traceability.
    replan_flag: Optional[bool] # Set by the executor to indicate that the planner should revise the plan.
    replan_attempts: Optional[Dict[int, Dict[int, int]]] # Replan attempts tracked per step number.
    final_answer: Optional[str] # The final synthesized answer

**Note**: `State` inherits from `MessagesState`, which is defined with a single `messages` key that keeps track of the list of messages shared among agents. So in addition to the fields you just defined for `State`, it also has a `messages` field from `MessagesState`. 

## 2.2 Create planner

The planner takes in the user's query and generates a plan. The plan consists of a sequence of numbered steps; each step includes the action and the sub-agent that is assigned to that action.

In [4]:
import json
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langgraph.types import Command

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def planner_node(state: State) -> Command:
    """Creates or revises a plan based on the user query."""
    
    user_query = state.get("user_query", "")
    replan_flag = state.get("replan_flag", False)
    current_step = state.get("current_step", 0)
    enabled_agents = state.get("enabled_agents", [])
    
    # Build the planning prompt
    system_prompt = f"""
    You are a planning agent. Create a step-by-step plan to answer the user's query.
    Available agents: {enabled_agents}
    
    Return your plan as a JSON list where each step has:
    - "step": step number (integer)
    - "agent": which agent to use
    - "action": what the agent should do
    
    Example format:
    [
        {{"step": 1, "agent": "web_researcher", "action": "Search for information about..."}},
        {{"step": 2, "agent": "chart_generator", "action": "Create a chart showing..."}}
    ]
    """
    
    if replan_flag:
        system_prompt += f"\n\nThis is a replan. The previous plan failed at step {current_step}. Please revise the plan."
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"User query: {user_query}")
    ]
    
    response = llm.invoke(messages)
    
    try:
        # Extract JSON from the response
        content = response.content
        # Find JSON in the response
        import re
        json_match = re.search(r'\[.*\]', content, re.DOTALL)
        if json_match:
            plan = json.loads(json_match.group())
        else:
            plan = json.loads(content)
    except:
        # Fallback plan if parsing fails
        plan = [
            {"step": 1, "agent": "web_researcher", "action": f"Search for: {user_query}"},
            {"step": 2, "agent": "synthesizer", "action": "Summarize the findings"}
        ]
    
    print(f"Planner created plan: {plan}")
    
    return Command(
        update={
            "plan": plan,
            "current_step": 1,
            "replan_flag": False,
            "messages": [HumanMessage(content=f"Plan created: {plan}", name="planner")]
        },
        goto="executor"
    )

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

## 2.3 Create executor

In [5]:
from langgraph.graph import END

def executor_node(state: State) -> Command:
    """Executes the current step in the plan."""
    
    plan = state.get("plan", [])
    current_step = state.get("current_step", 1)
    user_query = state.get("user_query", "")
    
    # Find the current step in the plan
    current_step_data = None
    for step_dict in plan:
        if step_dict.get("step") == current_step:
            current_step_data = step_dict
            break
    
    if not current_step_data:
        # No more steps, go to synthesizer
        return Command(
            update={
                "agent_query": f"Synthesize the answer for: {user_query}",
                "messages": [HumanMessage(content="All steps completed", name="executor")]
            },
            goto="synthesizer"
        )
    
    agent = current_step_data.get("agent")
    action = current_step_data.get("action")
    
    print(f"Executor: Step {current_step} - Agent: {agent}, Action: {action}")
    
    return Command(
        update={
            "agent_query": action,
            "current_step": current_step + 1,
            "messages": [HumanMessage(content=f"Executing step {current_step}: {action}", name="executor")]
        },
        goto=agent
    )

## 2.4 Create web researcher

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

def web_research_node(state: State) -> Command:
    """Performs web research using Tavily."""
    
    agent_query = state.get("agent_query", "")
    
    print(f"Web Researcher: Searching for: {agent_query}")
    
    try:
        # Initialize Tavily search
        search_tool = TavilySearchResults(max_results=5)
        results = search_tool.invoke(agent_query)
        
        # Format results
        formatted_results = "Search Results:\n"
        for i, result in enumerate(results, 1):
            formatted_results += f"\n{i}. {result.get('content', '')}\n"
            if 'url' in result:
                formatted_results += f"   Source: {result['url']}\n"
        
        print(f"Web Researcher found {len(results)} results")
        
    except Exception as e:
        formatted_results = f"Web search failed: {str(e)}. Using mock data."
        # Mock data for testing without API key
        formatted_results = """Search Results (Mock Data):
        1. JPMorgan Chase - Market Cap: $550 billion
        2. Bank of America - Market Cap: $280 billion  
        3. Wells Fargo - Market Cap: $200 billion
        4. Goldman Sachs - Market Cap: $130 billion
        5. Morgan Stanley - Market Cap: $150 billion
        """
    
    return Command(
        update={
            "messages": [HumanMessage(content=formatted_results, name="web_researcher")]
        },
        goto="executor"
    )

## 2.5 Create chart generator

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from io import StringIO

def chart_node(state: State) -> Command:
    """Generates charts based on data from previous steps."""
    
    agent_query = state.get("agent_query", "")
    messages = state.get("messages", [])
    
    print(f"Chart Generator: Creating chart for: {agent_query}")
    
    # Extract data from previous messages
    data_text = ""
    for msg in messages[-5:]:  # Look at last 5 messages
        if hasattr(msg, 'content'):
            data_text += msg.content + "\n"
    
    # Try to extract numerical data
    banks = []
    market_caps = []
    
    # Pattern to extract bank names and market caps
    patterns = [
        r'([\w\s]+)\s*[-–]\s*Market Cap:\s*\$([\d.]+)\s*(billion|trillion)',
        r'([\w\s]+):\s*\$([\d.]+)\s*(billion|trillion)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, data_text, re.IGNORECASE)
        if matches:
            for match in matches:
                bank_name = match[0].strip()
                value = float(match[1])
                unit = match[2].lower()
                if unit == 'trillion':
                    value *= 1000
                banks.append(bank_name)
                market_caps.append(value)
            break
    
    # If no data extracted, use default data
    if not banks:
        banks = ['JPMorgan Chase', 'Bank of America', 'Wells Fargo', 'Goldman Sachs', 'Morgan Stanley']
        market_caps = [550, 280, 200, 130, 150]
    
    # Create the chart
    plt.figure(figsize=(12, 6))
    colors = plt.cm.Blues(np.linspace(0.4, 0.8, len(banks)))
    bars = plt.bar(banks, market_caps, color=colors)
    
    plt.title('Market Capitalization of Top US Banks', fontsize=16, fontweight='bold')
    plt.xlabel('Bank', fontsize=12)
    plt.ylabel('Market Cap (Billion USD)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, value in zip(bars, market_caps):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
                f'${value}B', ha='center', va='bottom', fontsize=10)
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    
    # Save the chart
    chart_path = '/tmp/market_cap_chart.png'
    plt.savefig(chart_path, dpi=100, bbox_inches='tight')
    plt.show()
    
    chart_description = f"Chart created showing market capitalization for {len(banks)} banks. "
    chart_description += f"Top bank: {banks[market_caps.index(max(market_caps))]} with ${max(market_caps)}B market cap."
    
    print(f"Chart Generator: Chart saved to {chart_path}")
    
    return Command(
        update={
            "messages": [HumanMessage(content=chart_description, name="chart_generator")]
        },
        goto="chart_summarizer"
    )

## 2.6 Create chart summarizer

In [8]:
def chart_summary_node(state: State) -> Command:
    """Summarizes the chart that was generated."""
    
    messages = state.get("messages", [])
    
    # Get the chart description from the previous step
    chart_info = ""
    for msg in reversed(messages):
        if hasattr(msg, 'name') and msg.name == "chart_generator":
            chart_info = msg.content
            break
    
    if not chart_info:
        chart_info = "No chart information available"
    
    # Create a summary using the LLM
    summary_prompt = f"""
    Summarize this chart information in 2-3 sentences:
    {chart_info}
    
    Focus on the key insights and trends shown in the data.
    """
    
    response = llm.invoke([HumanMessage(content=summary_prompt)])
    summary = response.content
    
    print(f"Chart Summarizer: {summary}")
    
    return Command(
        update={
            "messages": [HumanMessage(content=summary, name="chart_summarizer")]
        },
        goto="executor"
    )

## 2.7 Create synthesizer

In [9]:
def synthesizer_node(state: State) -> Command:
    """Synthesizes all information into a final answer."""
    
    user_query = state.get("user_query", "")
    messages = state.get("messages", [])
    
    # Collect all relevant messages
    relevant_msgs = []
    for msg in messages:
        if hasattr(msg, 'name') and msg.name in ['web_researcher', 'chart_generator', 'chart_summarizer']:
            relevant_msgs.append(f"{msg.name}: {msg.content}")
    
    synthesis_instructions = """
    Create a comprehensive answer to the user's question based on all the information gathered.
    Include:
    - Key findings from web research
    - Data visualizations created
    - Main insights and conclusions
    
    Keep the response concise but informative.
    """
    
    summary_prompt = [
        HumanMessage(content=(
            f"User question: {user_query}\n\n"
            f"{synthesis_instructions}\n\n"
            f"Context:\n\n" + "\n\n---\n\n".join(relevant_msgs)
        ))
    ]
    
    llm_reply = llm.invoke(summary_prompt)
    answer = llm_reply.content.strip()
    
    print(f"\n=== FINAL ANSWER ===\n{answer}\n===================")
    
    return Command(
        update={
            "final_answer": answer,
            "messages": [HumanMessage(content=answer, name="synthesizer")],
        },
        goto=END
    )

## 2.8 Build the agent graph

In [ ]:
from langgraph.graph import START, StateGraph

workflow = StateGraph(State)
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("web_researcher", web_research_node)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("chart_summarizer", chart_summary_node)
workflow.add_node("synthesizer", synthesizer_node)

workflow.add_edge(START, "planner")

graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph nodes:", list(workflow.nodes.keys()))

## 2.9 Use the agent

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ </b> The following two queries might take <b>2-5 minutes</b> to output the results.</p>

<div style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 
<p>🚨 &nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video. For example: 

In the first query, the agent might decide to call the synthesizer instead of calling the chart generator. In this case, you will only see text summarizing the web search results instead of a chart, which means the agent's answer is not completely relevant to the user's query. This is what you'll learn how to evaluate in the next lessons.
</p>

In [ ]:
from langchain.schema import HumanMessage
import json

query = "Chart the current market capitalization of the top 5 banks in the US?"
print(f"Query: {query}")
print("="*50)

state = {
    "messages": [HumanMessage(content=query)],
    "user_query": query,
    "enabled_agents": ["web_researcher", "chart_generator", 
                       "chart_summarizer", "synthesizer"],
}

try:
    result = graph.invoke(state)
    print("\n" + "="*50)
    print("Final Answer:")
    print(result.get("final_answer", "No final answer generated"))
except Exception as e:
    print(f"Error running graph: {e}")

print("\n" + "="*50)

In [ ]:
query = "Identify current regulatory changes for the financial services industry in the US."
print(f"Query: {query}")
print("="*50)

state = {
    "messages": [HumanMessage(content=query)],
    "user_query": query,
    "enabled_agents": ["web_researcher", "chart_generator", 
                       "chart_summarizer", "synthesizer"],
}

try:
    result = graph.invoke(state)
    print("\n" + "="*50)
    print("Final Answer:")
    print(result.get("final_answer", "No final answer generated"))
except Exception as e:
    print(f"Error running graph: {e}")

print("\n" + "="*50)